<a href="https://www.kaggle.com/code/alisahial/fitbit-cleaned-data-part-2?scriptVersionId=160693248" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### 5.4 User Types by Activity Levels <a class="anchor" id="usertypes1"></a>
The CDC recommends 150 minutes of physical activity each week. (https://www.cdc.gov/physicalactivity/basics/adults/index.htm) 

So I wanted to see the sum of the average minutes of Very Active, Fairly Active & Lightly active minutes to see if each distinct Id was hitting those CDC activity recommendations.
#### 5.4.1 User Types by Active Minutes <a class="anchor" id="usertypes2"></a>

In [ ]:
# SELECT Id, 
#  AVG(very_active_minutes) + AVG(fairly_active_minutes) + AVG(lightly_active_minutes) AS Total_Avg_Active_Minutes,
#  CASE 
#   WHEN AVG(very_active_minutes) + AVG(fairly_active_minutes) + AVG(lightly_active_minutes) >= 150 THEN 'Meets CDC Recommendation'
#   WHEN AVG(very_active_minutes) + AVG(fairly_active_minutes) + AVG(lightly_active_minutes) <150 THEN 'Does Not Meet CDC Recommendation'
#   END CDC_Recommendations
# FROM `my-data-project-2208-389311.fitbit_project.daily_activity`
# GROUP BY Id
# ORDER BY Id;

**CONCLUSION** : On average, 27 of the 33 users met the CDC Activity Length Recommendations, while 6 did not. Resulting in 81% meeting the CDC guidelines. (*note CDC guidelines were last updates on June 2nd, 2022)

#### 5.4.2 User Types by Total Steps <a class="anchor" id="usertypes3"></a> 

A Healthline.com article (“How many steps do I need a day?”) written by Sara Lindberg in 2019 cited a 2011 study by Tudor-Locke et. al. titled “How many steps/day are enough? for adults” which found that 10,000 steps/day is a reasonable target for healthy adults. Lindberg (2019) breaks down activity level by steps into three categories based off the 2011 study by Tudor-Locke et. al.: 

* **Inactive:** less than 5,000 steps per day 
* **Average (somewhat active):** ranges from 7,500 to 9,999 steps per day
* **Very Active:**  more than 12,500 steps per day

Sources: [Healthline Article](https://www.healthline.com/health/how-many-steps-a-day#How-many-steps-for-weight-loss?) | [2011 Study](https://ijbnpa.biomedcentral.com/articles/10.1186/1479-5868-8-79#Sec3![image.png](attachment:164e6137-441b-4400-b1e1-d07f2268749b.png)

In [ ]:
# SELECT Id,
#  ROUND(avg(total_steps),2) AS Avg_Total_Steps,
#  CASE
#   WHEN avg(total_steps) <= 5000 THEN 'Inactive'
#   WHEN avg(total_steps) BETWEEN 5001 AND 7499 THEN 'Low Active User'
#   WHEN avg(total_steps) BETWEEN 7500 AND 9999 THEN 'Average Active User'
#   WHEN avg(total_steps) BETWEEN 10000 AND 12499 THEN 'Active User'
#   WHEN avg(total_steps) >= 12500 THEN 'Very Active User'
#   END User_Type
# FROM `fluid-tuner-406721.FitBit_Case_Study.daily_activity`
# GROUP BY Id
# ORDER BY User_Type;

**Conclusion**

* Inactive User: 8 users
* Low Active User: 9 users
* Average Active User: 9 users
* Active User: 5 users
* Very Active User: 2 users

Only 21% of the users have > 10,000 steps a day


Diving deeper into inactive users: Intrigued by users averaging less than 5,000 steps. I filtered out the 8 individuals that from the daily_sleep.csv. I created a new table with them and labeled it "inactiveusers_sleep". From there, I found the total avergage time asleep, time in bed as well as total time wasted.


In [ ]:
# -- daily_sleep file 

# SELECT * 
# FROM `fluid-tuner-406721.FitBit_Case_Study.daily_sleep` 
# WHERE ID IN (1844505072,1927972279,4057192912,4445114986,6775888955,8792009665,2320127002,4020332650)

# -- inactiveusers_sleep file

# SELECT ROUND(AVG(total_minutes_asleep), 2) AS MINS_ASLEEP_AVG,
#   ROUND(AVG(total_minutes_bed), 2) AS TOTAL_TIME_IN_BED_AVG
# FROM `fluid-tuner-406721.FitBit_Case_Study.inactiveusers_sleep`

Avg. Minutes asleep: 401.05 minutes

Avg. Time in bed: 440.7 minutes

Avg. Time wasted: 39.65 minutes

**Conclusion**: The avg minutes asleep is less than the recommended amount of 8 hours of sleep

Lets look at daily activities of inactive users:

In [ ]:
# -- Inactive users - daily_activities

# SELECT  *
# FROM `fluid-tuner-406721.FitBit_Case_Study.daily_activity` 
# WHERE ID IN (1844505072,1927972279,4057192912,4445114986,6775888955,8792009665,2320127002,4020332650);


# -- AVG intensities for inactive users 

# SELECT AVG(very_active_minutes) AS Avg_Very_Active_Minutes,
#  AVG(fairly_active_minutes) AS Avg_fairly_active_minutes,
#  AVG(lightly_active_minutes) AS avg_lightly_active_minutes,
#  AVG(sedentary_minutes) AS avg_sedentary_minutes
# FROM `fluid-tuner-406721.FitBit_Case_Study.inactiveusers_dailyactivities` 


# -- FITIBT NOT USED by inactive users

# SELECT COUNT(ID) AS NOT_USED
# FROM `fluid-tuner-406721.FitBit_Case_Study.inactiveusers_dailyactivities` 
# WHERE sedentaryminutes = 1440

AVG_very_active_minutes: 3.6 minutes

AVG_fairly_active_minutes: 4.1 minutes

AVG_lightly_active_minutes: 111.7 minutes

AVG_sedentary_minutes: 1166.2 minutes 

Total times FitBit not used: 55


**CONCLUSION** 

* 1. Sedentary activities seems to higher

* 2. Looking at the data of people with 1440 minutes of sedentary activity it seems that they signify users who have the fitness band but didnt use it that day , this pattern occurring in 25% of the data of inactive users suggests that these users leave there Fitibt at home a lot.

### 5.5 Total calories v/s steps <a class="anchor" id="steps"></a>

Next, I wanted to see relation between total calories burned & various intensity of activities. 

In [ ]:
# SELECT Id, 
#  SUM(sedentary_minutes) AS sedentary_min,
#  SUM(lightly_active_minutes) AS lightly_active_mins, 
#  SUM(fairly_active_minutes) AS fairly_active_mins, 
#  SUM(very_active_minutes) AS very_active_mins,
#  SUM(total_steps) AS total_steps,
#  SUM(calories) AS total_calories
# FROM `fluid-tuner-406721.FitBit_Case_Study.daily_activity` 
# GROUP BY Id

**CONCLUSION** :All three shows, as expected an increase in calories burned with the increase in level of activity.  Total calories expenditure rises as the total number of steps are rising per user

### 5.6 Weekday-wise avg intensity of activity <a class="anchor" id="weekdayintensity"></a> 

I also looked at weekday & weekend difference between avg intensity of activity minues.

In [ ]:
# SELECT activity_day,
#  ROUND (avg(very_active_minutes), 2) AS Avg_Very_Active_Minutes,
#  ROUND (avg(fairly_active_minutes), 2) AS Avg_Fairly_Active_Minutes,
#  ROUND (avg(lightly_active_minutes), 2) AS Avg_Lightly_Active_Minutes,
#  ROUND (avg(sedentary_minutes), 2) AS Avg_Sedentary_Minutes
# FROM `fluid-tuner-406721.FitBit_Case_Study.daily_activity`
# GROUP BY activity_day

**CONCLUSION**

The following trends are intresting to note:

* 1. Very active minutes, fairly active minutes & Lightly active minutes rise a little during Saturdays and similarly sedendary activity does fall a little. 
* 2. Sunday again we can see a fall in active minutes and rise in sedantary minutes. 

### 5.7 Average steps taken on different days of the week  <a class="anchor" id="avgsteps"></a>
Now of steps taken on an average with the corresponding day of the week.

In [ ]:
# SELECT activity_day,
#  ROUND (avg(total_steps), 2) AS Average_Total_Steps
# FROM `fluid-tuner-406721.FitBit_Case_Study.daily_activity`
# GROUP BY activity_day
# ORDER BY Average_Total_Steps DESC

**CONCLUSION** : * As expected, Saturdays witness the most steps, corroborating our earlier observations about a general rise in activity intensity during saturdays.

### 5.8 Desired time of workout & Sleep <a class="anchor" id="time"></a>
It would be intresting to look at the desired time of workout based on amount of calories burned by all users. 

In [ ]:
# --Desired time of workout based on most calories burned in day

# SELECT activity_hour,
#  ROUND(AVG(calories),2) AS total_calories_by_hour
# FROM `fluid-tuner-406721.FitBit_Case_Study.hourly_calories` 
# GROUP BY activity_hour

 **CONCLUSION**
* 5 PM -9 PM is the duration of highest calories burned and hence could be the desired time of workout for all the users combined on an average.

In [ ]:
# -- AVG steps taken per hour

# SELECT  activity_hour,
#   ROUND(AVG(total_steps), 2) AS AVG_STEPS_BY_HOUR
# FROM `fluid-tuner-406721.FitBit_Case_Study.hourly_steps` 
# GROUP BY activity_hour
# ORDER BY activity_hour ASC

**CONCLUSION** : Analyzing users' step counts unveils steps starts rising after 5pm. Indicating that is when users are begining to workout

### 5.9  Sleep Analysis <a class="anchor" id="deeper"></a>

Questions to answer:
1. What is the average sleep duration and quality for each user?
2. Are there any correlations between sleep patterns and other factors like activity levels
3. Sleep habits of the users and how it compares to activity level. 

First I looked at day wise avg minutes of sleep by all users.

In [ ]:
# --at which day all the users had the most hours of sleep

# SELECT 
#  ROUND(AVG(total_minutes_asleep)/60 ,2) AS Avg_Hours_Asleep, 
# activity_day 
# FROM `fluid-tuner-406721.FitBit_Case_Study.daily_sleep` AS SLEEP
# INNER JOIN fluid-tuner-406721.FitBit_Case_Study.daily_activity AS ACTIVITY 
#   ON ACTIVITY.id = SLEEP.id
#   GROUP BY ACTIVITY_DAY

[Google Sheet Link](https://docs.google.com/spreadsheets/d/13l0PHVQJ9E6eOwv0PAsTQLKNth_29U-f2QN9tXKLspw/edit#gid=710813922)

**CONCLUSION** : Day-wise average sleep showed no trends, with users averaging just about 7 hours consistently.

Next, I looked into the time spent in bed awake. 

The equation used in order to find 'wasted time in bed' = Total minutes in bed - Total minutes asleep 

In [ ]:
# SELECT id, ROUND(AVG(total_minutes_asleep),2) as avg_sleep_time_min,
#  ROUND(Avg(total_minutes_bed),2) as avg_time_bed_min,
#  ROUND(AVG(total_minutes_bed - total_minutes_asleep),2) as wasted_bed_time_min
# FROM `fluid-tuner-406721.FitBit_Case_Study.daily_sleep` 
# GROUP BY id

**CONCLUSION**: Calculating "wasted time" in bed (minutes in bed - minutes asleep) provided insights into sleep quality. Notably, most users showed low "wasted time," suggesting efficient sleep cycles and potentially good sleep quality.

Next we see the Relationship between avg minutes of sleep and calories burned per user

In [ ]:
# SELECT ACTIVITY.Id, 
#   ROUND(avg(ACTIVITY.total_steps),2) AS AvgTotalSteps,
#    ROUND(avg(ACTIVITY.calories),2) AS AvgCalories,
#    ROUND(avg(SLEEP.total_minutes_asleep),2) AS AvgTotalMinutesAsleep
# FROM `fluid-tuner-406721.FitBit_Case_Study.daily_activity` AS ACTIVITY
#   INNER JOIN fluid-tuner-406721.FitBit_Case_Study.daily_sleep AS SLEEP
#   ON ACTIVITY.id = SLEEP.id
# GROUP BY ACTIVITY.Id

**CONCLUSION**: The relationship between People who slept better and burned calories seems to be positive!

### 5.10  METs analysis  <a class="anchor" id="METs"></a>

* Q. What is METs?
     * MET: The ratio of the work metabolic rate to the resting metabolic rate. One MET is defined as 1 kcal/kg/hour. The **metabolic equivalent of task (MET)** is the objective measure of the ratio of the rate at which a person expends energy equivalent to the energy cost of sitting down.

* MET values here are divided by 10 because according to meta data of the dataset: "All MET values exported from Fitabase are multiplied by 10. Please divide by 10 to get accurate MET values"

--METs

SELECT DISTINCT temp1.id, 
    temp1.activity_minute, 
    SUM(temp1.METs)/24 as avg_mets, 
    temp2.Calories
FROM `fluid-tuner-406721.FitBit_Case_Study.METs_minute` as temp1
INNER JOIN fluid-tuner-406721.FitBit_Case_Study.daily_activity AS temp2
ON temp1.id = temp2.id 
Group By temp1.Id, temp1.activity_minute, temp2.Calories
Order by Id,activity_minute;

![](http://)

**CONCLUSION**: Our data reveals a positive link between METs (activity level) and calorie burn.

## <font color='#E2A76F'>6. ACT Phase</font> <a class="anchor" id="act"></a>

### 6.1 Conclusion <a class="anchor" id="conclusion"></a> 

The global fitness tracker market size was valued at 47 billion dollar in 2022 & is projected to grow from  53.94 billion dollar  in 2023 to $182.90 billion by 2030. To succeed in this competitive market, the fitness band company needs to have a strong marketing strategy. By diving into Fitbit user data, we gain invaluable insights into real-world fitness watch usage. This unveils both market gaps and user preferences, empowering us to tailor Bellabeat and highlight features that truly resonate with the audience. The most important data points is as follows:

* 1. _High engagement_: 64% of users logged data for the entire data time period of 31 days. This shows that users are actively tracking their progress.

* 2. _Potential for improvement_: 8% of non-usage (72 missed wearings) indicates room for improvement in encouraging consistent wear. Tuesday emerges as the day with the most missed wearings. 

* 3. _CDC Activity Length Recommendations_:  On average, 27 of the 33 users met the CDC Activity Length Recommendations, while 6 did not (Very Active, Fairly Active & Lightly active minutes together). 

* 4. _High average sedentary minutes_: Average sedentary minutes of 931.78 minutes (15 hours 31.78 minutes). The average sedentary time might be higher than recommended guidelines. For instance, some health organizations recommend limiting sitting time to less than 8 hours per day. 81% of the time by all users was spent on sedentary time hence they possibly have a less active lifestyle.
 
* 5.  Very active minutes, fairly active minutes & Lightly active minutes rise a little during Saturdays and similarly sedendary activity does fall a little suggesting users are more likely to engage in physical activities on weekends compared to weekdays. Weekends present an opportunity to encourage and sustain healthy activity levels. By delving deeper into the reasons behind this trend and tailoring interventions accordingly, you can help users maintain a more active lifestyle throughout the week.
 
* 6. 5 PM -7 PM is the duration of highest calories burned and hence could be the desired time of workout for all the users combined on an average.
  
* 7. The relationship between People who slept better and burned calories seems to be positive. Users averaging around 7 hours of sleep consistently. 

### 6.2 Recommendations <a class="anchor" id="recommendations"></a>

**_1. Digital Marketting_**:

   **A. Partner with influencers to promote Ivy health tracker**: As seen in the above analysis, all 33 users have logged in  step, activity instensity and calorie burned features on Fitbit, Bellabeat can effectively reach its target audience by partnering with relevant influencers: a fitness expert and a registered dietitian. They can also offer exclusive discounts and promotions to influencers' followers. Schedule social media posts and email campaigns for early morning audiences.
   
   
   **B. Run targeted advertising campaigns on social media and search engines**: The data tells one thing: That 81% of the time by all users was spent on sedentary time hence they possibly have a desk job. Therefore we can Craft targeted ads showcasing Bellabeat's features like activity reminders and step challenges as the antidote to sedentary desk jobs. Target platforms frequented by professional women on Facebook, LinkedIn, and fitness-focused Instagram accounts.
   
  **C.Create engaging social media content**: Since social media is the prefered mode of advertisment for Bellabeat. We can feature real Bellabeat users sharing their fitness journeys, showcasing product features, and offering helpful tips. Feature inspiring stories of women overcoming challenges and achieving their goals with Bellabeat by their side.
   
   **D. Search Engine Optimization (SEO)** : Optimize BellaBeat's website and content for relevant keywords like "women's fitness tracker," "desk job wellness," and " menstrual cycle recorder", "pregnancy-safe workouts" to reach potential customers organically.
   
 **_2. Embrace Heart and Weight_** : Analyze the underutilized features like heart rate and weight tracking. Consider developing content, challenges, or integrations focused on these aspects to cater to a wider range of user interests and goals which could be a possible market gap. 

**_3.Segmentation and Targeting_**:

   **A. Data insights reveal a key opportunity: some customers bought the fitbit band but haven't fully embraced its potential. For such existing customers following approach can be followed**

* 1. _Personalized nudges_: Trigger targeted emails highlighting the benefits of increased activity based on individual user data. Showcase how small, attainable steps lead to big health improvements.
* 2. _Playful motivators_: Send gentle app notifications reminding users to move, offering playful challenges based on their inactivity patterns. Celebrate milestones with virtual high-fives and encourage progress with fun rewards.
* 3. _Sleep for Strength_: Leverage sleep data to send personalized app notifications with sleep coaching tips. Emphasize the link between quality sleep and boosted energy, improved activity levels, and overall well-being.
* 4. _Challenge them to shine_: Design bite-sized nightly sleep challenges, like achieving a specific sleep duration or maintaining a consistent sleep schedule, to encourage healthy sleep habits in a fun and engaging way.

**B. Position the band as the "after-work empowerment tool"**: Since Post-office workouts seem to be popular amongst our sample group we can frame evening workouts as a way to de-stress, boost energy, and reclaim control after a busy day. 
      
**_4. Emphasize data and insights_** : Fitbit is known for its data-driven approach to fitness tracking. The brand uses data to provide users with insights into their activity levels, sleep patterns, and overall health. Bellabeat can differentiate itself by offering even more advanced data and insights, such as personalized fitness plans, health recommendations, and integration with other health apps.

**_5. Community Connection_** : Foster a vibrant online community through social media groups, interactive forums, and virtual workout sessions. Encourage users to share tips, support each other, and celebrate achievements together.

**_6. Product Development and Improvement_**:
  * A. Our data shows the sample users as early risers. Develop features specifically for early mornings, like customizable sunrise alarms, guided meditation sessions for stress reduction, or benefit of a quick workout routines for before work.
  * B. Weekly progress report with segregation of weekday-weekend reports can be given to users to keep them motivated for the whole week. 

**Additional Considerations**
* 1. Data privacy: Ensure you comply with all data privacy regulations and protect user data.
* 2. Ethical marketing: Avoid using misleading or deceptive marketing practices.
* 3. Transparency: Be transparent about your data collection and usage practices.